### movieCd와 genreCd를 매칭시켜 영화의 장르를 표기하기 위해 데이터를 전처리하고 MovieGenre 테이블에 데이터를 넣는 작업
##### 작업자: 이종택

In [2]:
#패키지 준비

import pymysql
import pandas as pd

In [3]:
#csv 파일을 DataFrame형식으로 불러와서 MySQL에 들어갈 수 있도록 nan -> None 변환 및 데이터 속성 변경(int -> str) 후 list형식으로 변환

movie_list_result = pd.read_csv("..\data-files\movie_list_result.csv", encoding='utf-8')
movie_list_result = movie_list_result.where((pd.notnull(movie_list_result)), None)
movie_list_result['movieCd'] = movie_list_result['movieCd'].astype('int').astype('str')
movie_list_result['prdtYear'] = movie_list_result['prdtYear'].astype('int').astype('str')
movie_list_result['openDt'] = movie_list_result['openDt'].astype('int').astype('str')

print(movie_list_result.info())

print("==========================")
s = movie_list_result.values.tolist()
print(s[0])


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 47051 entries, 0 to 47050
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   movieCd      47051 non-null  object
 1   directors    47051 non-null  object
 2   movieNm      47051 non-null  object
 3   movieNmEn    47051 non-null  object
 4   prdtYear     47051 non-null  object
 5   openDt       47051 non-null  object
 6   typeNm       46994 non-null  object
 7   prdtStatNm   46913 non-null  object
 8   nationAlt    46962 non-null  object
 9   repNationNm  46968 non-null  object
 10  genreAlt     47051 non-null  object
 11  repGenreNm   47051 non-null  object
dtypes: object(12)
memory usage: 4.3+ MB
None
['20206061', '콜린 트레보로우', '쥬라기 월드: 도미니언', 'Jurassic World: Dominion', '2022', '0', '장편', '개봉예정', '미국', '미국', '액션,어드벤처', '액션']


In [70]:
# movie_list_result 안의 genreAlt 속의 장르 묶음을 ','을 기준으로 나눠준 후 새로운 칼럼(genreAlt2)을 만들어서 리스트로 넣음 ex) '액션,어드벤쳐' -> ['액션', '어드벤쳐']

test = movie_list_result
test["genreAlt2"] = test["genreAlt"].map(lambda x : x.split(","))
print(len(test))

print(test['genreAlt2'])
print("==========================================================")
print(test['genreAlt2'][0])
print("==========================================================")
print(test['genreAlt2'][0][0])


47051
0        [액션, 어드벤처]
1         [액션, 스릴러]
2           [다큐멘터리]
3        [코미디, 드라마]
4           [다큐멘터리]
            ...    
47046         [드라마]
47047       [다큐멘터리]
47048         [드라마]
47049         [드라마]
47050         [드라마]
Name: genreAlt2, Length: 47051, dtype: object
['액션', '어드벤처']
액션


In [53]:
# movieCd와 복수의 genreNm을 각각의 리스트로 나누는 작업 ex) [20206061, ['액션','드라마']] --> [20206061, '액션'], [20206061, '드라마']

repo = []
genre_matching = []
movie_order = 0

for movie in test['movieCd']:

    if movie == test.iloc[movie_order]['movieCd']:
        print(movie, test.iloc[movie_order]['movieCd'])

        for genre in test.iloc[movie_order]['genreAlt2']:
            repo = [movie, genre]
            genre_matching.append(repo)
            continue
        movie_order += 1
        
    else:
        continue

20206061 20206061
20210547 20210547
20211413 20211413
20212027 20212027
20212064 20212064
20212063 20212063
20212042 20212042
20212062 20212062
20212041 20212041
20136895 20136895
20205986 20205986
20211176 20211176
20218415 20218415
20210611 20210611
20211241 20211241
20210031 20210031
20196264 20196264
20210864 20210864
20181118 20181118
20219056 20219056
20206464 20206464
20189638 20189638
20210689 20210689
20148592 20148592
20136792 20136792
20201815 20201815
20212015 20212015
20211929 20211929
20210663 20210663
20195959 20195959
20211159 20211159
20183772 20183772
20211220 20211220
20210790 20210790
20209295 20209295
20211735 20211735
20199882 20199882
19838068 19838068
20010264 20010264
20210882 20210882
20194403 20194403
20211734 20211734
20205736 20205736
20211768 20211768
20188441 20188441
20138950 20138950
20218390 20218390
20217102 20217102
20217167 20217167
20210794 20210794
20215801 20215801
20210028 20210028
20210883 20210883
20199962 20199962
20211994 20211994
20211978 2

In [60]:
# 위에서 만든 리스트를 [genreCd, genreNm]과 매칭시키기 위해 데이터프레임 형식으로 전환

df = pd.DataFrame(genre_matching, columns=['movieCd', 'genreNm'])
df

,movieCd,genreNm
0,20206061,액션
1,20206061,어드벤처
2,20210547,액션
3,20210547,스릴러
4,20211413,다큐멘터리
...,...,...
73210,20070895,드라마
73211,20072121,다큐멘터리
73212,20060861,드라마
73213,20060849,드라마


In [55]:
# DB에서 genreCd, genreNm 가져와서 저장하는 작업

conn = pymysql.connect(host="localhost",
                       database="firstkino",
                       user="kdigital",
                       password="mysql",
                       charset="utf8")

all_genres = []
with conn.cursor() as cursor:
    cursor.execute("SELECT * FROM Genre")
    genres = cursor.fetchall()
    all_genres.append(genres)

conn.commit()

conn.close()

In [56]:
# genreCd, genreNm이 튜플형식으로 되어있으므로 list형식으로 변환 및 DataFrame 형식으로 변환

genre_list = list(all_genres[0])
print(genre_list)

genre_df = pd.DataFrame(genre_list, columns=['genreCd', 'genreNm'])

print(genre_df)

[(1, '판타지'), (2, '드라마'), (3, '애니메이션'), (4, '전쟁'), (5, 'SF'), (6, '스릴러'), (7, '미스터리'), (8, '멜로/로맨스'), (9, '공포(호러)'), (10, '공연'), (11, '기타'), (12, '범죄'), (13, '코미디'), (14, '뮤지컬'), (15, '성인물(에로)'), (16, '서부극(웨스턴)'), (17, '액션'), (18, '다큐멘터리'), (19, '사극'), (20, '어드벤처'), (21, '가족')]
    genreCd   genreNm
0         1       판타지
1         2       드라마
2         3     애니메이션
3         4        전쟁
4         5        SF
5         6       스릴러
6         7      미스터리
7         8    멜로/로맨스
8         9    공포(호러)
9        10        공연
10       11        기타
11       12        범죄
12       13       코미디
13       14       뮤지컬
14       15   성인물(에로)
15       16  서부극(웨스턴)
16       17        액션
17       18     다큐멘터리
18       19        사극
19       20      어드벤처
20       21        가족


In [61]:
# df와 genre_df를 genreNm을 기준으로 병합

result = pd.merge(df, genre_df, on='genreNm')

In [66]:
# movieCd를 기준으로 정렬하여 보기 위한 코드

result.sort_values(by=['movieCd'], axis=0)

,movieCd,genreNm,genreCd
69629,19158001,전쟁,4
38466,19158001,드라마,2
70156,19158001,서부극(웨스턴),16
18334,19188001,코미디,13
64460,19190002,공포(호러),9
...,...,...,...
70581,20219978,기타,11
25311,20219984,드라마,2
18143,20219993,코미디,13
25307,20219993,드라마,2


In [ ]:
# DB에 저장하기 위해 list형식으로 병환

result_list = result.values.tolist()

In [69]:
# list형식의 데이터를 테이블에 넣는 작업

conn = pymysql.connect(host="localhost",
                       database="firstkino",
                       user="kdigital",
                       password="mysql",
                       charset="utf8")

with conn.cursor() as cursor:
    for post in result_list:
        cursor.execute("INSERT INTO MovieGenre (movieCd, genreCd) values (%s, %s)", (post[0], post[2]))
    conn.commit()

conn.close()